In [4]:
# IMPORT MODULES
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from tensorflow.contrib.layers import flatten
from tf.keras.datasets import fashion_mnist
from tf.keras.layers import Convolution2D, Input, Activation, Flatten, Dense
from tf.keras.optimizers import Adam

# Network Diagram
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
%matplotlib inline

ImportError: dynamic module does not define module export function (PyInit__tf2)

In [ ]:
# LOAD DATA
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print("X train shape:", x_train.shape)
print("Y train shape:", y_train.shape)
print("X test shape:", x_test.shape)
print("Y test shape:", y_test.shape)

# Reshape the data (num_sample, image_width, image_height, channel)
# This format is what CNN layer expects the input to be in.
num_train_sample = x_train.shape[0]
num_test_sample = x_test.shape[0]
image_width = x_train.shape[1]
image_height = x_train.shape[2]
channel = 1
num_class = 10

x_train.reshape(num_train_sample, image_width, image_height, channel)
x_test.reshape(num_test_sample, image_width, image_height, channel)

# One hot encoding
y_train = keras.utils.to_categorical(y_train, num_class)
y_test = keras.utils.to_categorical(y_test, num_class)

In [ ]:
# Shape after one hot encoding
print("X train shape:", x_train.shape)
print("Y train shape:", y_train.shape)
print("X test shape:", x_test.shape)
print("Y test shape:", y_test.shape)

In [ ]:
# DATA EXPLORATION
# Load a random image
index = random.randint(0, num_train_sample)
image = x_train[index]
image = image.reshape((image_width, image_height))

# Visualize the image
plt.figure(figsize = (5, 5))
plt.imshow(image, cmap="gray")
print(y_train[index])

In [ ]:
# CREATE MODEL
from keras.models import Model
from keras.layers.convolutional import Conv2D

# Conv2D Block: returns tensor output
def Conv2DBlock(layer_input, num_filters, kernel_size):
    layer = Conv2D(filters=num_filters,
                   kernel_size=(kernel_size, kernel_size),
                   kernel_initializer="he_normal",
                   padding="same")(layer_input)
    layer = Activation("relu")(layer)
    return layer

# Network Function: returns model[input, output]
def Network(network_input, num_filters):
    c1 = Conv2DBlock(network_input, num_filters=num_filters, kernel_size=3)
    c2 = Conv2DBlock(c1, num_filters=num_filters * 2, kernel_size=3)
    # Fully Connected Layer: output = (num_sample, 10)
    f = Flatten()(c2)
    d = Dense(num_class)(f)
    model = Model(inputs=[network_input], outputs=[d])
    return model

In [ ]:
input_img = Input((28, 28, 1), name='img')
model = Network(input_img, num_filters=32)

In [ ]:
model.summary()

In [ ]:
# Visualize network
plot_model(model, to_file='model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
# Setup model (specify optimizer, loss function and metrics)
model.compile(optimizer = Adam(), loss = 'mean_squared_error', metrics = ["accuracy", "binary_accuracy", "mse"])

print("X train shape:", x_train.shape)
print("Y train shape:", y_train.shape)

In [ ]:
results = model.fit(x_train, y_train, batch_size=16, epochs=5, verbose=2)

In [ ]:
sample = x_train[0]
sample = sample.reshape(1, 28, 28, 1)

print(model.predict(sample))
print(y_train[0])

In [ ]:
model.save("03-16-2019-model.h5")